First, run the initializer script below


In [ ]:
import pandas as pd
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

worksheet = gc.open('LSD Contact Info Sheet 21-22 ').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
# Convert to a DataFrame and render.
df = pd.DataFrame.from_records(rows)

worksheet = gc.open('input').sheet1
rows = worksheet.get_all_values()
inputdf = pd.DataFrame.from_records(rows)
inputdf.rename(columns = {0:"first name", 1:"last name"}, inplace = True)

#col_names = df.iloc[0]
col_names = pd.Series(data=["timestamp","first name", "pref first name", "last name", "grade", "period", "level", "email", "phone", "address", "p1name", "p1email", "p1phone", "p2name", "p2email", "p2phone"])
df.rename(columns=col_names, inplace = True)
df.drop(0,inplace=True)
df.head()

Then, define the parameter columns.


In [ ]:
contact_info = df.values.tolist()
class Student:
  def __init__(self, info_pack):
    for ind in range(1,len(info_pack)):
      if len(info_pack[ind]) > 0:
        while info_pack[ind][-1] == " " and len(info_pack[ind]) > 1:
          info_pack[ind] = info_pack[ind][:-1]
    self.first_name = info_pack[1]
    self.pref_first_name = info_pack[2]
    self.last_name = info_pack[3]
    #index 4 is full name
    self.grade = info_pack[5]
    self.period = info_pack[6]
    self.level = info_pack[7]
    self.email = info_pack[8]
    self.phone = info_pack[9]
    self.address = info_pack[10]
    self.p1name = info_pack[11]
    self.p1email = info_pack[12]
    self.p1phone = info_pack[13]
    self.p2name = info_pack[14]
    self.p2email = info_pack[15]
    self.p2phone = info_pack[16]
  def return_val(self, param):
    if param == "first name":
      return self.first_name
    if param == "pref first name":
      return self.pref_first_name
    if param == "last name":
      return self.last_name
    if param == "grade":
      return self.grade
    if param == "period":
      return self.period
    if param == "level":
      return self.level
    if param == "phone":
      return self.phone
    if param == "email":
      return self.email
    if param == "address":
      return self.address
    if param == "p1name":
      return self.p1name
    if param == "p1email":
      return self.p1email
    if param == "p1phone":
      return self.p1phone
    if param == "p2name":
      return self.p2name
    if param == "p2email":
      return self.p2email
    if param == "p2phone":
      return self.p2phone
      
students = []
for item in contact_info:
  students.append(Student(item)) 

In [ ]:
!pip install fuzzywuzzy
from fuzzywuzzy import process
inputs = []
old_inputs = inputdf.values.tolist()
for item in old_inputs:
  if process.extract("Anahita Kashani Gregg",[item[0]])[0][1]>80: #hard codes around the ppl with double last/first names
    inputs.append(["Anahita", "Kashani Gregg"])
  elif process.extract("Gieve Kashani Gregg",[item[0]])[0][1]>80:
    inputs.append(["Gieve", "Kashani Gregg"])
  elif process.extract("Min Joo Kim",[item[0]])[0][1]>80:
    inputs.append(["Min Joo", "Kim"])
  else:
    inputs.append(item[0].rsplit(" ",1))

len(inputs)


/usr/local/lib/python3.7/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


31

In [ ]:
#cell that u should tweak

params = [ "email", "phone"] #change this accordingly
outputs = []
for first,last in inputs:
  output_row = [first,last]
  myStudent = "???"
  for student in students:
    if process.extract(student.first_name,[first])[0][1] > 80 and process.extract(student.last_name,[last])[0][1] > 80:
    #if first == student.first_name and last == student.last_name:
      myStudent = student
      break
    elif student.pref_first_name != "":
      if process.extract(student.pref_first_name,[first])[0][1] > 80 and process.extract(student.last_name,[last])[0][1] > 80: 
        myStudent = student
        break
  for param in params:
    if myStudent == "???":
      output_row.append("err")
    else:
      output_row.append(myStudent.return_val(param))
  outputs.append(output_row)
len(outputs)
    

31

In [ ]:
outputdf = pd.DataFrame(data=outputs)

worksheet = gc.open('output').sheet1
my_file = outputdf.to_csv()

from google.colab import drive
drive.mount('/content/drive')

with open('/content/drive/MyDrive/lsd-machine-input-output/output.csv', 'w') as f:
  f.write(my_file)


Mounted at /content/drive
